# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5a88e96ac0>
├── 'a' --> tensor([[ 0.5030,  0.1414,  0.8699],
│                   [ 0.1882,  0.6535, -0.7220]])
└── 'x' --> <FastTreeValue 0x7f5a88e96430>
    └── 'c' --> tensor([[-0.1826,  0.6593,  0.0802, -1.6962],
                        [-0.3041,  1.1822,  0.1079,  0.1236],
                        [ 1.6155,  0.6978, -0.0491,  0.7802]])

In [4]:
t.a

tensor([[ 0.5030,  0.1414,  0.8699],
        [ 0.1882,  0.6535, -0.7220]])

In [5]:
%timeit t.a

57.5 ns ± 3.61 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5a88e96ac0>
├── 'a' --> tensor([[-1.3999, -0.4025, -1.2515],
│                   [ 0.7182, -0.8130,  1.8546]])
└── 'x' --> <FastTreeValue 0x7f5a88e96430>
    └── 'c' --> tensor([[-0.1826,  0.6593,  0.0802, -1.6962],
                        [-0.3041,  1.1822,  0.1079,  0.1236],
                        [ 1.6155,  0.6978, -0.0491,  0.7802]])

In [7]:
%timeit t.a = new_value

68.8 ns ± 0.0307 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5030,  0.1414,  0.8699],
               [ 0.1882,  0.6535, -0.7220]]),
    x: Batch(
           c: tensor([[-0.1826,  0.6593,  0.0802, -1.6962],
                      [-0.3041,  1.1822,  0.1079,  0.1236],
                      [ 1.6155,  0.6978, -0.0491,  0.7802]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5030,  0.1414,  0.8699],
        [ 0.1882,  0.6535, -0.7220]])

In [11]:
%timeit b.a

51.8 ns ± 0.044 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0725,  1.3122, -0.5543],
               [ 0.8493,  0.7631,  0.2013]]),
    x: Batch(
           c: tensor([[-0.1826,  0.6593,  0.0802, -1.6962],
                      [-0.3041,  1.1822,  0.1079,  0.1236],
                      [ 1.6155,  0.6978, -0.0491,  0.7802]]),
       ),
)

In [13]:
%timeit b.a = new_value

438 ns ± 0.111 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

826 ns ± 1.24 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 22.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 466 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 336 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f59dadb08b0>
├── 'a' --> tensor([[[ 0.5030,  0.1414,  0.8699],
│                    [ 0.1882,  0.6535, -0.7220]],
│           
│                   [[ 0.5030,  0.1414,  0.8699],
│                    [ 0.1882,  0.6535, -0.7220]],
│           
│                   [[ 0.5030,  0.1414,  0.8699],
│                    [ 0.1882,  0.6535, -0.7220]],
│           
│                   [[ 0.5030,  0.1414,  0.8699],
│                    [ 0.1882,  0.6535, -0.7220]],
│           
│                   [[ 0.5030,  0.1414,  0.8699],
│                    [ 0.1882,  0.6535, -0.7220]],
│           
│                   [[ 0.5030,  0.1414,  0.8699],
│                    [ 0.1882,  0.6535, -0.7220]],
│           
│                   [[ 0.5030,  0.1414,  0.8699],
│                    [ 0.1882,  0.6535, -0.7220]],
│           
│                   [[ 0.5030,  0.1414,  0.8699],
│                    [ 0.1882,  0.6535, -0.7220]]])
└── 'x' --> <FastTreeValue 0x7f59dadb04f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 59.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f59dadb02b0>
├── 'a' --> tensor([[ 0.5030,  0.1414,  0.8699],
│                   [ 0.1882,  0.6535, -0.7220],
│                   [ 0.5030,  0.1414,  0.8699],
│                   [ 0.1882,  0.6535, -0.7220],
│                   [ 0.5030,  0.1414,  0.8699],
│                   [ 0.1882,  0.6535, -0.7220],
│                   [ 0.5030,  0.1414,  0.8699],
│                   [ 0.1882,  0.6535, -0.7220],
│                   [ 0.5030,  0.1414,  0.8699],
│                   [ 0.1882,  0.6535, -0.7220],
│                   [ 0.5030,  0.1414,  0.8699],
│                   [ 0.1882,  0.6535, -0.7220],
│                   [ 0.5030,  0.1414,  0.8699],
│                   [ 0.1882,  0.6535, -0.7220],
│                   [ 0.5030,  0.1414,  0.8699],
│                   [ 0.1882,  0.6535, -0.7220]])
└── 'x' --> <FastTreeValue 0x7f5a88e92370>
    └── 'c' --> tensor([[-0.1826,  0.6593,  0.0802, -1.6962],
                        [-0.3041,  1.1822,  0.1079,  0.1236],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 625 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.1 µs ± 93.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.1826,  0.6593,  0.0802, -1.6962],
                       [-0.3041,  1.1822,  0.1079,  0.1236],
                       [ 1.6155,  0.6978, -0.0491,  0.7802]],
              
                      [[-0.1826,  0.6593,  0.0802, -1.6962],
                       [-0.3041,  1.1822,  0.1079,  0.1236],
                       [ 1.6155,  0.6978, -0.0491,  0.7802]],
              
                      [[-0.1826,  0.6593,  0.0802, -1.6962],
                       [-0.3041,  1.1822,  0.1079,  0.1236],
                       [ 1.6155,  0.6978, -0.0491,  0.7802]],
              
                      [[-0.1826,  0.6593,  0.0802, -1.6962],
                       [-0.3041,  1.1822,  0.1079,  0.1236],
                       [ 1.6155,  0.6978, -0.0491,  0.7802]],
              
                      [[-0.1826,  0.6593,  0.0802, -1.6962],
                       [-0.3041,  1.1822,  0.1079,  0.1236],
                       [ 1.6155,  0.6978, -0.0491,  0.7802]],

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 95.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.1826,  0.6593,  0.0802, -1.6962],
                      [-0.3041,  1.1822,  0.1079,  0.1236],
                      [ 1.6155,  0.6978, -0.0491,  0.7802],
                      [-0.1826,  0.6593,  0.0802, -1.6962],
                      [-0.3041,  1.1822,  0.1079,  0.1236],
                      [ 1.6155,  0.6978, -0.0491,  0.7802],
                      [-0.1826,  0.6593,  0.0802, -1.6962],
                      [-0.3041,  1.1822,  0.1079,  0.1236],
                      [ 1.6155,  0.6978, -0.0491,  0.7802],
                      [-0.1826,  0.6593,  0.0802, -1.6962],
                      [-0.3041,  1.1822,  0.1079,  0.1236],
                      [ 1.6155,  0.6978, -0.0491,  0.7802],
                      [-0.1826,  0.6593,  0.0802, -1.6962],
                      [-0.3041,  1.1822,  0.1079,  0.1236],
                      [ 1.6155,  0.6978, -0.0491,  0.7802],
                      [-0.1826,  0.6593,  0.0802, -1.6962],
                   

In [28]:
%timeit Batch.cat(batches)

145 µs ± 420 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

305 µs ± 14.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
